In [6]:
import pandas as pd 
import splitfolders

from keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers
from keras import optimizers

In [3]:
#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

Copying files: 5856 files [00:01, 4003.95 files/s]


In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 3747 images belonging to 2 classes.


In [4]:
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='binary')

Found 936 images belonging to 2 classes.


In [5]:
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='binary')

Found 1173 images belonging to 2 classes.
